**<파일 경로>**
- C:/Users/Brian/Desktop/Data Science/Projects/Playdata/dataset/made_by_전처리/train
- C:/Users/Brian/Desktop/Data Science/Projects/Playdata/dataset/made_by_전처리/test

In [1]:
train_path = input()

C:/Users/Brian/Desktop/Data Science/Projects/Playdata/dataset/made_by_전처리/train


In [2]:
test_path = input()

C:/Users/Brian/Desktop/Data Science/Projects/Playdata/dataset/made_by_전처리/test


In [3]:
import numpy as np
import pandas as pd
import datetime as dt
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# seaborn 폰트 및 스타일 설정
sns.set(font = "Malgun Gothic", rc = {"axes.unicode_minus": False}, style = 'whitegrid')
# Pandas 출력 결과 부동소수점 설정
pd.options.display.float_format = '{:.1f}'.format

from sklearn.model_selection import KFold, cross_val_score, cross_validate, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from bayes_opt import BayesianOptimization

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

- 먼저 메모리 절약을 위해 각 변수들의 데이터 타입을 지정해 준 뒤, 각 데이터 셋을 불러오도록 하겠다.

In [4]:
df_train_dtype = {
    # 지하철 데이터 관련 변수들 데이터 타입 지정
    'boarding_num': np.int32, 'log_boarding_num': np.float32, 'platform_area': np.float32, 
    'length': np.float32, 'transfer_num': np.float32, 'entrance': np.float32, 
    'msi_cnt': np.float32, 'hsi_cnt': np.float32, 'uvi_cnt': np.float32, 
    'hpi_cnt': np.float32, 'mti_cnt': np.float32, 'day': np.int32,
    # 인구 데이터 관련 변수들 데이터 타입 지정
    'pop_below_20': np.float32, 'pop_20_30_40': np.float32, 'pop_over_50': np.float32,
    # 기상 데이터 관련 변수들 데이터 타입 변경
    'temp': np.float32, 'windspeed': np.float32, 'rain': np.float32
}

df_test_dtype = {
    # 지하철 데이터 관련 변수들 데이터 타입 지정
    'boarding_num': np.int32, 'platform_area': np.float32, 
    'length': np.float32, 'transfer_num': np.float32, 'entrance': np.float32, 
    'msi_cnt': np.float32, 'hsi_cnt': np.float32, 'uvi_cnt': np.float32, 
    'hpi_cnt': np.float32, 'mti_cnt': np.float32, 'day': np.int32,
    # 인구 데이터 관련 변수들 데이터 타입 지정
    'pop_below_20': np.float32, 'pop_20_30_40': np.float32, 'pop_over_50': np.float32,
    # 기상 데이터 관련 변수들 데이터 타입 변경
    'temp': np.float32, 'windspeed': np.float32, 'rain': np.float32
}

In [5]:
df_train = pd.read_csv('{}/df_train_linear_line2.csv'.format(train_path), dtype = df_train_dtype)
df_test = pd.read_csv('{}/df_test_linear_line2.csv'.format(test_path), dtype = df_test_dtype)

- 추가적인 메모리 절약을 위해 line과 station_name 변수의 타입을 category로 변환하고, 원-핫 인코딩 처리된 변수들의 dtype을 모두 uint8로 지정해주겠다.

In [6]:
cat_vars = ['line', 'station_name']

for var in tqdm(cat_vars):
    df_train[var] = df_train[var].astype('category')
    df_test[var] = df_test[var].astype('category')

In [7]:
# One-Hot Encoding 처리된 변수들의 데이터 타입 변경
df_train[df_train.columns[21:]] = df_train.iloc[:, 21:].astype('uint8')
df_test[df_test.columns[20:]] = df_test.iloc[:, 20:].astype('uint8')

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460000 entries, 0 to 1459999
Columns: 120 entries, date to season_winter
dtypes: category(2), float32(16), int32(2), object(1), uint8(99)
memory usage: 252.0+ MB


In [9]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364000 entries, 0 to 363999
Columns: 119 entries, date to season_winter
dtypes: category(2), float32(15), int32(2), object(1), uint8(99)
memory usage: 61.4+ MB


# Modeling

## 평가 지표
- 이번 분석에서는 RMSE를 평가 지표로 사용할 것이다.
- 따라서 RMSE를 계산해주는 함수를 하나 만들도록 하겠다.

<참고>
- 이와 더불어 R2 score 또한 서브 평가 지표로써 사용하도록 하겠다.

In [10]:
def rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

## 변수 선택 및 데이터 분할

### Feature dataset / Target dataset 생성
- 먼저 train dataset과 test dataset에서 모델에 넣지 않을 불필요한 변수들은 제거해주겠다.
    - train dataset
        - 기본적으로 제외시킨 변수들
            - 'date', 'line', 'station_name', 'boarding_num', 'log_boarding_num', 'day'
        - 예측 성능을 비교해서 제외시킨 변수들
            - 'pop_over_50'
                - 앞서 수행한 EDA 결과를 바탕으로, pop_over_50(50세 이상 인구 수) 변수는 다중공선성 문제로 인해 모델의 예측 성능을 저하시킨다고 판단했으므로 모델에 넣어주지 않겠다.
    - test dataset
        - 기본적으로 제외시킨 변수들
            - 'date', 'line', 'station_name', 'boarding_num', 'day'
        - 예측 성능을 비교해서 제외시킨 변수들
            - 'pop_over_50'
                - 앞서 수행한 EDA 결과를 바탕으로, pop_over_50(50세 이상 인구 수) 변수는 다중공선성 문제로 인해 모델의 예측 성능을 저하시킨다고 판단했으므로 모델에 넣어주지 않겠다.

In [11]:
df_train.columns.tolist()

['date',
 'line',
 'station_name',
 'boarding_num',
 'platform_area',
 'length',
 'transfer_num',
 'entrance',
 'msi_cnt',
 'hsi_cnt',
 'uvi_cnt',
 'hpi_cnt',
 'mti_cnt',
 'temp',
 'windspeed',
 'rain',
 'pop_below_20',
 'day',
 'pop_20_30_40',
 'pop_over_50',
 'log_boarding_num',
 'boarding_승차',
 'boarding_하차',
 'gu_강남구',
 'gu_강동구',
 'gu_강북구',
 'gu_강서구',
 'gu_관악구',
 'gu_광진구',
 'gu_구로구',
 'gu_금천구',
 'gu_노원구',
 'gu_도봉구',
 'gu_동대문구',
 'gu_동작구',
 'gu_마포구',
 'gu_서대문구',
 'gu_서초구',
 'gu_성동구',
 'gu_성북구',
 'gu_송파구',
 'gu_양천구',
 'gu_영등포구',
 'gu_용산구',
 'gu_은평구',
 'gu_종로구',
 'gu_중구',
 'gu_중랑구',
 'type_복합식',
 'type_상대식',
 'type_섬식',
 'floor_1F',
 'floor_1FB2',
 'floor_1FB3',
 'floor_1FB5',
 'floor_2F',
 'floor_2FB2',
 'floor_2FB3',
 'floor_3F',
 'floor_4F',
 'floor_5FB2',
 'floor_B1',
 'floor_B2',
 'floor_B3',
 'floor_B4',
 'floor_B5',
 'floor_B6',
 'floor_B8',
 'year_2018',
 'year_2019',
 'year_2020',
 'month_1',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',


In [12]:
# columns for drop in train dataset
drop_list1 = ['date', 'line', 'station_name', 'boarding_num', 
              'log_boarding_num', 'day', 'pop_over_50']
# columns for drop in test dataset
drop_list2 = ['date', 'line', 'station_name', 
              'boarding_num', 'day', 'pop_over_50']

# 학습 데이터 셋
X_features = df_train.drop(drop_list1, axis = 1, inplace = False)
y_target = df_train['boarding_num'] # 로그 변환되지 않은 타겟 변수
log_y_target = df_train['log_boarding_num'] # 로그 변환된 타겟 변수
print('학습 데이터 셋:', X_features.shape, y_target.shape, log_y_target.shape)

# 테스트 데이터 셋
X_test = df_test.drop(drop_list2, axis = 1, inplace = False)
y_test = df_test['boarding_num']
print('테스트 데이터 셋:', X_test.shape, y_test.shape)

학습 데이터 셋: (1460000, 113) (1460000,) (1460000,)
테스트 데이터 셋: (364000, 113) (364000,)


### 학습 데이터 셋 분할
- 전체 학습 데이터 셋을 다시 학습 데이터 셋과 검증 데이터 셋으로 분할해주겠다.

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(X_features, log_y_target, 
                                                      test_size = 0.2, random_state = 1021)
print('학습 및 검증 데이터 셋:', X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

학습 및 검증 데이터 셋: (1168000, 113) (292000, 113) (1168000,) (292000,)


## Baseline Model
- 선형 모델
    1. 선형 회귀
    2. 릿지 회귀
    3. 라쏘 회귀
    4. 엘라스틱넷 회귀

In [14]:
# def baseline_linear_model(model):
#     kfold = KFold(n_splits = 5)
    
#     scores = cross_val_score(model, X_features, log_y_target, 
#                              scoring = make_scorer(rmse),
#                              cv = kfold)
#     mean_rmse = np.mean(scores)
#     scores_std = scores.std()

#     print('###', model.__class__.__name__, '###')
#     print('교차 검증별 RMSE: ', np.round(scores, 4))
#     print('평균 검증 RMSE: ', np.round(mean_rmse, 4))
#     print('교차 검증별 RMSE 표준편차: ', np.round(scores_std, 4))
#     print('-' * 80)

In [15]:
def baseline_linear_model(model):
    # 모델 학습
    model.fit(X_train, y_train)
    # 예측
    log_y_train_pred = model.predict(X_train)
    log_y_valid_pred = model.predict(X_valid)

    # 예측값이 로그 변환된 target 기반으로 학습되어 예측됐으므로, np.expm1()을 이용하여 스케일 변환
    y_train_pred = np.expm1(log_y_train_pred)
    y_valid_pred = np.expm1(log_y_valid_pred)

    # 학습/검증 데이터 셋의 target 값 또한 로그 변환됐으므로, np.expm1()을 이용하여 원래 스케일로 변환
    y_train_exp = np.expm1(y_train)
    y_valid_exp = np.expm1(y_valid)

    # RMSE 및 R2 Score 계산
    rmse_train = rmse(y_train_exp, y_train_pred)
    r2_score_train = r2_score(y_train_exp, y_train_pred)
    rmse_valid = rmse(y_valid_exp, y_valid_pred)
    r2_score_valid = r2_score(y_valid_exp, y_valid_pred)

    print('###', model.__class__.__name__, '###')
    print('학습 데이터 셋에 대한 RMSE: ', np.round(rmse_train, 4))
    print('학습 데이터 셋에 대한 R2 Score: ', np.round(r2_score_train, 4))
    print('검증 데이터 셋에 대한 RMSE: ', np.round(rmse_valid, 4))
    print('검증 데이터 셋에 대한 R2 Score: ', np.round(r2_score_valid, 4))
    print('-' * 50)

In [16]:
lr = LinearRegression()
baseline_linear_model(lr)

### LinearRegression ###
학습 데이터 셋에 대한 RMSE:  1295.8062
학습 데이터 셋에 대한 R2 Score:  0.5457
검증 데이터 셋에 대한 RMSE:  1290.223
검증 데이터 셋에 대한 R2 Score:  0.5486
--------------------------------------------------


In [17]:
# alphas = [0.01, 0.1, 1, 10, 100]

# ridge = Ridge(random_state = 1021)
# lasso = Lasso(random_state = 1021)
# elastic = ElasticNet(random_state = 1021)

# linear_models = [ridge, lasso, elastic]

# for model in tqdm(linear_models):
#     baseline_nonlinear_model(model)

- 선형 회귀 모델의 결과를 보면, valid dataset에 대한 예측 성능뿐만 아니라 train dataset에 대한 예측 성능도 좋지 않음을 알 수 있다.
    - 따라서 Ridge, Lasso와 같은 규제 모델들을 사용하는 것이 의미가 없어보인다. (실제로 사용해봤으나 예측 성능이 좋아지지 않았다)
    - train dataset에서 조차 좋지 않은 성능을 보이므로, 더 복잡한 모델을 사용하는 것이 좋을 것 같다.
- 따라서 다른 비선형 모델들을 활용하여 예측을 수행하는 방향으로 분석을 진행하도록 하겠다.
    1. 결정 트리
    2. 랜덤포레스트
    3. XGBoost
    4. LightGBM